In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from tqdm.notebook import tqdm
import torchtext
import torchdata
import keras
import tensorflow

/Users/maksimkuznetsov/Documents/Учёба/w2v/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [10]:
from tensorflow.keras.datasets import imdb

In [19]:
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)

# Получение словаря слов
word_index = imdb.get_word_index()

(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data()

word_index = imdb.get_word_index()

start_char = 1
oov_char = 2
index_from = 3

inverted_word_index = dict(
    (i + index_from, word) for (word, i) in word_index.items()
)

inverted_word_index[start_char] = "[START]"
inverted_word_index[oov_char] = "[OOV]"

train_set = np.array([" ".join(inverted_word_index[i] for i in item) for item in training_data])
test_set = np.array([" ".join(inverted_word_index[i] for i in item) for item in testing_data])
                     
print(train_set.shape, test_set.shape)

(25000,) (25000,)


In [13]:
train_set

array(["[START] this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing

In [27]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maksimkuznetsov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/maksimkuznetsov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/maksimkuznetsov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/maksimkuznetsov/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [28]:
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = list(filter(str.isalpha, word_tokenize(text.lower())))
    text = list(lemmatizer.lemmatize(token) for token in text)
    text = list(word for word in text if word not in stopwords)
    return ' '.join(text)
train_set_prepor = [preprocess_text(seq) for seq in train_set]
test_set_prepor = [preprocess_text(seq) for seq in test_set]

In [30]:
train_set_prepor[100]

'start great fan david lynch everything made dvd except hotel room hour twin peak movie found immediately grabbed bunch crudely drawn black white cartoon loud foul mouthed unfunny maybe know good maybe bunch crap wa foisted public name david lynch make buck let make clear care foul language part keep adjusting sound neighbor might highly disappointing release may well left deluxe box set curiosity highly recommend spend money'

Напишем baseline с log_reg для дальнейшего сравнения 

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [34]:
tf_idf = TfidfVectorizer().fit(train_set_prepor)
train_baseline = tf_idf.fit_transform(train_set_prepor)
test_baseline = tf_idf.transform(test_set_prepor)

In [35]:
log_reg = LogisticRegression(n_jobs=-1).fit(train_baseline, training_targets)

In [36]:
print('Train acc: ', accuracy_score(training_targets, log_reg.predict(train_baseline)))
print('Test acc: ', accuracy_score(testing_targets, log_reg.predict(test_baseline)))

Train acc:  0.93084
Test acc:  0.88048
